Analysis of TOPAS-nBio simulations of cells with nanoparticles

This notebook analyses the results of the simulations of TOPAS and TOPAS-nbio of cells with nanoparticles irradiated with I125 radiation source.

In [1]:
import sys
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint

# Import our custom modules
from dnadamage_phsp_manager import *
import sddparser
from chemistry_output_manager import *
from phsp_manager import count_phsp_particles
from topas_statistics import TOPASStatistics
from collections import defaultdict
from topas_csv_files_manager import process_csv_file, process_original_hists

## Notebook Functions

This notebook processes the results of multiple TOPAS simulations of cells with nanoparticles. It uses the following imported functions:

- `process_csv_file`: Processes TOPAS CSV files and extracts sum values and histories
- `process_original_hists`: Extracts history values from TOPAS simulation files
- `count_phsp_particles`: Counts particles in phase space files

These functions are organized in the appropriate module files:
- `topas_csv_files_manager.py`: Contains functions for processing TOPAS CSV files
- `phsp_manager.py`: Contains functions for processing phase space files

The notebook processes multiple runs and aggregates results for:
- Original histories
- Dose to nucleus (physical and chemical phases)
- Energy deposited in the cell
- Nanoparticle electron emissions

In [2]:
# Multirun processing
nruns=100 
#runs_filebases = [f'/home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run{i+1}/' for i in range(nruns)]
runs_filebases = [f'../TOPAS_CellsNPs/work/only_results_CellColony-med1-cell1/cell1/run{i+1}/' for i in range(nruns)]

Cell_results_files = {
    'Original_hists': 'DoseToCell_I125Beam.csv',
    'DoseToNucl_ph2': 'DoseNucleus_Total.csv',
    'DoseToNucl_ph3': 'DoseNucleus_Ph3.csv',
    'Ecell': 'EnergyToCell.csv',
    'NP_el': 'PhaseSpace_NP',
}

Cell_results = {
    'Original_hists': 0,
    'DoseToNucl_ph2': 0,
    'DoseToNucl_ph3': 0,
    'Ecell': 0,
    'NP_el': 0,
}

# Dictionary to store combined statistics for each result type
Cell_statistics = {}

# Process all runs
for run_idx, run_base in enumerate(runs_filebases):
    print(f"Processing run {run_idx+1}/{nruns}...", end='\r')
    
    # Process each result type
    for result_key, file_name in Cell_results_files.items():
        file_path = os.path.join(run_base, file_name)
        if result_key == 'NP_el':
            # For phase space files
            count, _ = count_phsp_particles(file_path)
            Cell_results[result_key] += count
        elif result_key == 'Original_hists':
            # Process Original_hists differently
            Cell_results[result_key] += process_original_hists(file_path)
        else:
            # For other CSV files
            stats = process_csv_file(file_path, return_stats_object=True)
            if not stats:  # If processing failed
                continue
                
            # Add or combine statistics
            if result_key in Cell_statistics:
                Cell_statistics[result_key] += stats
            else:
                Cell_statistics[result_key] = stats

print("\nProcessing complete!")

# Display results
print("\nResults Summary:")
print("-" * 50)

# First, display special cases
for key, value in Cell_results.items():
    if key == 'NP_el':
        print(f"{key}: {value} particles")
    elif key == 'Original_hists':
        print(f"{key}: {value} (total from {nruns} runs)")
        
print("\nMeasured Quantities:")
print("-" * 50)
# Then display statistics for measured quantities
for key, stats in Cell_statistics.items():
    print(f"\n{key}:")
    print(f"  {stats.measurement_type}:")
    print(f"  - Total: {stats.sum_value:.6e} {stats.units}")
    print(f"  - Mean per history: {stats.mean:.6e} {stats.units}/hist")
    print(f"  - Standard deviation per history: {stats.standard_deviation:.6e} {stats.units}/hist")
    print(f"  - Total histories: {stats.histories_with_scorer_active:,}")
    print(f"  - Count in bin: {stats.count_in_bin:,}")
    
    # Calculate and show both types of uncertainties:
    # 1. Standard error (uncertainty of the mean)
    std_error = stats.standard_deviation / np.sqrt(stats.histories_with_scorer_active)
    # 2. Run-to-run variation (as in merge_CellsNP_csv)
    unc_2sigma = 2 * std_error  # 2-sigma confidence interval
    
    print(f"  - Mean with uncertainty: {stats.mean:.6e} ± {unc_2sigma:.6e} {stats.units}/hist (2σ)")
    if stats.n_runs > 1:
        print(f"  - Run-to-run variation: {stats.run_variance:.6e} {stats.units}")

Processing run 100/100...
Processing complete!

Results Summary:
--------------------------------------------------
Original_hists: 60000000 (total from 100 runs)
NP_el: 3707 particles

Measured Quantities:
--------------------------------------------------

DoseToNucl_ph2:
  DoseToMedium:
  - Total: 2.974897e+00 Gy
  - Mean per history: 3.655323e-07 Gy/hist
  - Standard deviation per history: 3.902896e-05 Gy/hist
  - Total histories: 8,138,533
  - Count in bin: 1,494
  - Mean with uncertainty: 3.655323e-07 ± 2.736175e-08 Gy/hist (2σ)
  - Run-to-run variation: 1.641286e+00 Gy

DoseToNucl_ph3:
  DoseToMedium:
  - Total: 3.279188e+00 Gy
  - Mean per history: 7.672843e-07 Gy/hist
  - Standard deviation per history: 5.745658e-05 Gy/hist
  - Total histories: 4,273,759
  - Count in bin: 1,599
  - Mean with uncertainty: 7.672843e-07 ± 5.558591e-08 Gy/hist (2σ)
  - Run-to-run variation: 1.772489e+00 Gy

Ecell:
  EnergyDeposit:
  - Total: 2.076337e+01 MeV
  - Mean per history: 2.551243e-06 MeV/